# Analyse der potentiellen Solarenergie in der Schweiz
Dieses Notebook analysiert die potentiellen Solarenergiepotenziale in der Schweiz basierend auf Dach- und Fassadenflächen.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import requests

# Pfad zur JSON-Datei mit den Solarenergiepotenzialen
file_path_data = '../data/potentielle_solarenergie/Solarenergiepotenziale_Gemeinden_Daecher_und_Fassaden.json'

# Daten laden
data = pd.read_json(file_path_data)

# Daten in ein DataFrame umwandeln
df = pd.DataFrame(data)

# Daten nach Country = 'CH' filtern
df = df[df['Country'] == 'CH']

# Nur relevante Spalten auswählen
df = df[['Canton', 'Scenario1_RoofsOnly_PotentialSolarElectricity_GWh', 'Scenario3_RoofsFacades_PotentialSolarElectricity_GWh']]

# Erste Zeilen anzeigen
df.head()

## Balkendiagramm erstellen: Potenzielle Solarenergie in der Schweiz
Das Diagramm visualisiert die potentiellen Solarenergiepotenziale nach Kanton.

In [ ]:
# Plotly für die Visualisierung importieren
import plotly.graph_objects as go

# Daten für das gestapelte Balkendiagramm vorbereiten
stacked_data = df.groupby("Canton")[["Scenario1_RoofsOnly_PotentialSolarElectricity_GWh", "Scenario3_RoofsFacades_PotentialSolarElectricity_GWh"]].sum()

# Fassaden-Werte bereinigen (Fassaden - Dächer)
stacked_data.loc[:, "Scenario3_RoofsFacades_PotentialSolarElectricity_GWh"] -= stacked_data["Scenario1_RoofsOnly_PotentialSolarElectricity_GWh"]

# Sortieren der Daten in absteigender Reihenfolge
stacked_data = stacked_data.sort_values(by="Scenario1_RoofsOnly_PotentialSolarElectricity_GWh", ascending=False)

# Farben für die Top 4 Kantone definieren
colors_roofs = ['rgb(0, 0, 0)' if i < 4 else 'rgb(169, 169, 169)' for i in range(len(stacked_data))]
colors_facades = ['rgb(128, 128, 128)' if i < 4 else 'rgb(211, 211, 211)' for i in range(len(stacked_data))]

# Erstellen des gestapelten Balkendiagramms
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Dächer',
    x=stacked_data.index,
    y=stacked_data["Scenario1_RoofsOnly_PotentialSolarElectricity_GWh"],
    marker_color=colors_roofs,
    text=(stacked_data["Scenario1_RoofsOnly_PotentialSolarElectricity_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))
fig.add_trace(go.Bar(
    name='Fassaden',
    x=stacked_data.index,
    y=stacked_data["Scenario3_RoofsFacades_PotentialSolarElectricity_GWh"],
    marker_color=colors_facades,
    text=(stacked_data["Scenario3_RoofsFacades_PotentialSolarElectricity_GWh"] / 1000).round(1).astype(str) + 'k',
    textposition='inside'
))

# Layout anpassen
fig.update_layout(barmode='stack',
                  title='Potenzielle Solarenergie in der Schweiz nach Kanton (Stand März 2025)',
                  xaxis_title='Kanton',
                  yaxis_title='Potentielle Solarenergie (GWh)',
                  height=500,
                  template='simple_white',
                  legend_title_text='Flächenkategorie')

# Diagramm als HTML-Datei speichern
fig.write_html("../docs/assets/diagramme/pot_solarenergie_pro_kanton.html")

# Diagramm anzeigen
fig.show()